### Challenge 1

In [ ]:
# Use memoization to speed up calculation of repeated movement at the same street intersections
def memoize(f):
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper
    
north = [0,1]
south = [0,-1]
east = [1,0]
west = [-1,0]

def tourist(x):
    moved=[]
    for nsew in [north,south,east,west]:
        moved.append(tuple([x+y for x,y in zip(x,nsew)])) 
    return(moved)

memoize(tourist)

moves = 10
traveled = [(0,0)]

for n in range(moves):
    if n == 0:
        pos = tourist((0,0))
    else:
        pos = [tourist(coord) for coord in pos]
        pos = [item for sublist in pos for item in sublist]
        if n > 1 and n < 6:
            pos = [tup for tup in pos if tup[0]>1]
        traveled.extend(pos)

from math import sqrt

def crow(tup):
    return(sqrt(tup[0]**2 + tup[1]**2))

# After 10 moves, tourist is at least 3 blocks away from 0,0?
memoize(crow)
print(format(len([tup for tup in pos if crow(tup)>=3])/len(pos),'.10f'))
# 0.4539794922

# After 10 moves, probability tourist ever moved at least 5 city blocks away from 0,0?
# Store and check all values
print(format(len([tup for tup in traveled if crow(tup)>=5])/len(traveled),'.10f'))
# 0.0893814950

# After 10 moves, pos[0]<-1 while for previous 6 moves had pos[0]>1
# Takes at least 4 moves to go from w. 2nd ave to e. 2nd ave
print(format(len([tup for tup in pos if tup[0]<-1])/len(pos),'.10f'))
# 0.0014391447


In [ ]:
# Simulate movement of tourist to measure mean movements needed to reach >=10 blocks from start
import numpy as np
import time
import multiprocessing as mp


def memoize(f):
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper
    
north = [0,1]
south = [0,-1]
east = [1,0]
west = [-1,0]
compass = np.array([north,south,east,west], dtype=int)


def crow(tup):
    return np.linalg.norm(tup)

def summ(a):
    return a[0]+a[1]

crow = memoize(crow)
summ = memoize(summ)

sims = int(1e6)
randgen = np.random.randint(0,4,int(150*sims))

starttime = time.time()


def simul(n):
    pos = (0,0)
    count = 0
    while crow(pos) < 10:
        pos = tuple([summ((x,y)) for x,y in zip(pos,compass[randgen[n+count]])])
        count += 1
    return(count)


moves = np.array(simul(0), dtype=int)
with mp.Pool(processes=4) as pool:
    moves = np.append(moves, pool.map(simul, range(1,sims), 1))

print(format(np.mean(moves),'.10f'))
print(format(np.std(moves),'.10f'))

print('')
print('Elapsed time:', round(time.time()-starttime,1), 'seconds')
print('')

# Simulate tourist path many times count number of moves to reach 10 blocks
# After 10000 iterations and 39 seconds
# mean = 103.6451100000
# std = 71.5591338900

# Testing 1e4, 5e4, and 1e5 seems to show time elapsing linearly
# (3.5s, 17.8s, and 38.2s)
#  Can run simulation with more iterations

# Using 1e6 runs and 4 processes for 4 logical (2 physical) cores the time is 201s
# mean = 104.9390500609
# std = 74.0271578617
